<a href="https://colab.research.google.com/github/SajlaKM/Generate-Queries/blob/main/Generate_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Install important libraries

In [2]:
!pip install fitz nltk sumy transformers pdfminer.six torch

In [3]:
!pip install tools
!pip install pymupdf
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 19.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
Extract  PDF file

In [8]:
import pymupdf
import re
import nltk
from nltk.tokenize import sent_tokenize

import spacy
nlp = spacy.load("en_core_web_sm")

# Function to Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)
    text = ""

    for page in doc:
        text += page.get_text("text") + "\n"

    return text

# Load PDF file to be summarized
pdf_text = extract_text_from_pdf("Artificial Intelligence.pdf")
print(pdf_text[:1000])

# Preprocess Text
pdf_text = re.sub(r'\s+', ' ', pdf_text).strip()  # Remove extra spaces

doc = nlp(pdf_text)
sentences = [sent.text for sent in doc.sents]

print(f"\nExtracted {len(sentences)} sentences from the PDF!")

See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/307981242
Essay: Artiﬁcial Intelligence: Boon or Bane?
Article  in  SSRN Electronic Journal · August 2016
DOI: 10.2139/ssrn.2836438
CITATIONS
25
READS
225,383
1 author:
Amit Tyagi
Microsoft
3 PUBLICATIONS   25 CITATIONS   
SEE PROFILE
All content following this page was uploaded by Amit Tyagi on 06 October 2017.
The user has requested enhancement of the downloaded file.

   Artificial Intelligence: Boon or Bane? 
 
Amit Tyagi (Leibniz Universität Hannover) 
 
 
 
 
 
 
                Source: Elektor Magazine 
ABSTRACT 
 
Artificial Intelligence (AI) is transforming the nature of almost everything which is connected 
to human life e.g. employment, economy, communication, warfare, privacy, security, ethics, 
healthcare etc. However, we are yet to see its evolution in long-term, whether it’s leading 
humanity towards making this planet a better place to live or a place which is f

In [ ]:
Extractive Summary

In [5]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Function for Extractive Summarization using spaCy
def summarize_text_spacy(text, num_sentences=5):
    # Tokenize sentences using spaCy
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    # Extract the first few sentences
    summary = " ".join(sentences[:num_sentences])

    return summary

# Generate Extractive Summary
extractive_summary = summarize_text_spacy(pdf_text, num_sentences=10)

print(" Extractive Summary:\n", extractive_summary)


 Extractive Summary:
 See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/307981242 Essay: Artiﬁcial Intelligence: Boon or Bane? Article in SSRN Electronic Journal · August 2016 DOI: 10.2139/ssrn.2836438 CITATIONS 25 READS 225,383 1 author: Amit Tyagi Microsoft 3 PUBLICATIONS 25 CITATIONS SEE PROFILE All content following this page was uploaded by Amit Tyagi on 06 October 2017. The user has requested enhancement of the downloaded file. Artificial Intelligence: Boon or Bane? Amit Tyagi (Leibniz Universität Hannover) Source: Elektor Magazine ABSTRACT Artificial Intelligence (AI) is transforming the nature of almost everything which is connected to human life e.g. employment, economy, communication, warfare, privacy, security, ethics, healthcare etc. However, we are yet to see its evolution in long-term, whether it’s leading humanity towards making this planet a better place to live or a place which is full of disaster. Every techn

In [ ]:
Abstractive Summary

In [6]:
from transformers import pipeline

# Load Pre-trained Summarization Pipeline
summarizer = pipeline("summarization", model="t5-small")

# Function for Abstractive Summarization
def summarize_text_abstractive(text, max_length=150, min_length=50):
    summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    return summary[0]['summary_text']

# Generate Abstractive Summary
abstractive_summary = summarize_text_abstractive(pdf_text, max_length=150, min_length=50)

print("Abstractive Summary:\n", abstractive_summary)


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (8667 > 512). Running this sequence through the model will result in indexing errors


Abstractive Summary:
 if we keep ignoring the social bugs of AI, it could be a serious threat to humanity . it is important to have a centralized global governing body which can make sure that we can mitigate the consequences . if you are embracing AI, you will be able to create a system that can be used to solve problems .


In [ ]:
Summary of Each page

In [1]:
import pymupdf
from transformers import pipeline

# Load Pre-trained Summarization Model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Function to Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)  # Open PDF
    pages_text = []

    for i in range(len(doc)):  # Iterate through pages
        text = doc[i].get_text("text")
        if text.strip():  # Ignore empty pages
            pages_text.append(text)

    return pages_text

# Function to Split Long Text into Chunks
def split_text(text, max_words=400):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

# Function to Summarize Each Page
def summarize_pdf(pdf_path, start_page=2):  # Skip cover & contents (Page 0 & 1)
    pages_text = extract_text_from_pdf(pdf_path)
    summaries = {}

    for i in range(start_page, len(pages_text)):  # Start from actual content
        page_text = pages_text[i]

        if len(page_text.split()) > 50:  # Avoid summarizing very short pages
            text_chunks = split_text(page_text, max_words=400)  # Break large text
            summary_chunks = [summarizer(chunk, max_length=150, min_length=50, do_sample=False)[0]['summary_text'] for chunk in text_chunks]
            summary = " ".join(summary_chunks)  # Combine summaries
            summaries[f"Page {i+1}"] = summary  # Store summary

    return summaries

# Run the summarization on your PDF
pdf_path = "/content/Artificial Intelligence.pdf"  # Replace with your file path
page_wise_summaries = summarize_pdf(pdf_path, start_page=2)

# Print Summaries
for page, summary in page_wise_summaries.items():
    print(f"\n# {page} Summary:\n{summary}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
Your max_length is set to 150, but your input_length is only 58. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 150, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually,


# Page 3 Summary:
Current state of AI: ........................................................................................................................ 6 2.1 Techniques (Deep Learning) & Companies (IBM, Google, Facebook, Microsoft, OpenAI): 7 2.2 Next steps of AIs: ............................................................ 10 2.3 Problem related to AI:  ........................................................................................................................................... 102.3. Between AIs & Humans:  ............................................................................................. 11 2.4 Policies & Laws:  ..................................................................................................................... 13 2.5 Possible Solutions:   ................................................. 14 3. CONCLUSION: 14 4. BIBLIOGRAPHY: 15.

# Page 4 Summary:
Artificial Intelligence is intelligence exhibited by machines. Eric Schmidt, the execu

In [ ]:
Split text into sentences

In [9]:
import spacy

nlp = spacy.load("en_core_web_sm")

def split_into_sentences(text):
    """Splits text into meaningful sentences using spaCy"""
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

# Get sentences
sentences = split_into_sentences(pdf_text)
print(sentences[:5])  # Print first 5 sentences


['See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/307981242 Essay:', 'Artiﬁcial Intelligence: Boon or Bane?', 'Article in SSRN Electronic Journal · August 2016 DOI: 10.2139/ssrn.2836438 CITATIONS 25 READS 225,383 1 author: Amit Tyagi Microsoft 3 PUBLICATIONS 25 CITATIONS SEE PROFILE All content following this page was uploaded by Amit Tyagi on 06 October 2017.', 'The user has requested enhancement of the downloaded file.', 'Artificial Intelligence: Boon or Bane?']


In [ ]:
Extract important sentences to generate questions

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_key_sentences(sentences, top_n=5):
    """Extracts key sentences using TF-IDF"""
    vectorizer = TfidfVectorizer(stop_words="english")
    X = vectorizer.fit_transform(sentences)

    # Rank sentences by importance (sum of TF-IDF scores)
    scores = X.sum(axis=1).A1
    ranked_sentences = [sentences[i] for i in scores.argsort()[-top_n:][::-1]]

    return ranked_sentences

# Extract important sentences
key_sentences = extract_key_sentences(sentences, top_n=10)
print(key_sentences)


['Despite the fact that we are counting on Artificial Intelligence as the next tool to revolutionize the way we live, work and interact with each other -- which will be mostly enabled by machine-learning techniques – it remains unclear as to how these intelligent agents will help to solve more complex problems than the ones existing today (e.g. Poverty, Epidemics, climate changes) while keeping in mind that the state of the art in AI today is to intelligently recognize images and smartly playing games.', 'There is a question to whether individual learning machines will exchange in a cloud-like manner their knowledge and thus should be collectively law suited or individually [20].While this might be an issue to deal with in the remote future, coping mechanisms to handle job displacements and unequal capital access caused by the widening imbalance of labour and capital [16] are going to be needed way earlier.', 'There was a very good scenario presented in the science-fiction movie called

In [ ]:
Convert sentences to Questions

In [11]:
from transformers import pipeline

# Load the question-generation model
question_generator = pipeline("text2text-generation", model="t5-small")

def generate_questions(sentences):
    """Generate questions from important sentences"""
    questions = []
    for sentence in sentences:
        q = question_generator(f"generate question: {sentence}", max_length=50)
        questions.append(q[0]['generated_text'])
    return questions

# Generate questions
generated_questions = generate_questions(key_sentences)

# Display results
for i, (q, a) in enumerate(zip(generated_questions, key_sentences)):
    print(f"Q{i+1}: {q}")
    print(f"A{i+1}: {a}\n")


Device set to use cpu


Q1: AI
A1: Despite the fact that we are counting on Artificial Intelligence as the next tool to revolutionize the way we live, work and interact with each other -- which will be mostly enabled by machine-learning techniques – it remains unclear as to how these intelligent agents will help to solve more complex problems than the ones existing today (e.g. Poverty, Epidemics, climate changes) while keeping in mind that the state of the art in AI today is to intelligently recognize images and smartly playing games.

Q2: question
A2: There is a question to whether individual learning machines will exchange in a cloud-like manner their knowledge and thus should be collectively law suited or individually [20].While this might be an issue to deal with in the remote future, coping mechanisms to handle job displacements and unequal capital access caused by the widening imbalance of labour and capital [16] are going to be needed way earlier.

Q3: iRobot
A3: There was a very good scenario presente

In [ ]:
Save Queries and Answers into a json file

In [13]:
import json

# Create a dictionary of Q&A
qa_data = [{"question": q, "answer": a} for q, a in zip(generated_questions, key_sentences)]

# Save to JSON file
with open("qa_pairs.json", "w") as f:
    json.dump(qa_data, f, indent=4)

print("Q&A pairs saved to qa_pairs.json")


Q&A pairs saved to qa_pairs.json
